In [5]:
import numpy as np
sgm = lambda x: 1/(1+np.exp(-x))
nrand = np.random.random
zerov = np.zeros

In [ ]:
class RBM:
    '''A restricted Boltzmann machine model.'''
    def __init__(self, n_variables, n_hidden, spin = False):
        self.a = zerov(n_variables)
        self.b = zerov(n_hidden)
        self.w = zerov((n_variables,n_hidden))
        self.min = -1*spin # Evaluates to zero if the model is not spin-based

    def activate(self, v, w, b, gap = 1):
        '''Turns the neurons of a forward layer on, according to a sigmoid probability function.'''
        vec = w@v + b # The neurons evaluation
        l = len(vec) # sample size
        prob = sgm(gap*vec)
        out = np.full(l,self.min)
        out[prob < nrand(l)] = 1
        return out
    
    def grad(self):
        pass

    def train(self, data, n_epochs = 300, batch_size = 30, learning_rate = 0.1):
        '''Trains the model as a '''
        for epoch in range(n_epochs):
            continue